In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from collections import defaultdict
import hashlib
import os
import sys

In [ ]:
destination_db = "/content/drive/My Drive/IRCMS_GAN_collaborative_database/Experiments/colab-violingan/archon-analysis" #@param {type:"string"}
des_datasize = "if known" #@param {type:"string"}
min_size = "176440" #@param {type:"string"}

min_size = int(min_size)


In [ ]:
# OPTIONAL
# dest_datasize = len(
    # [name for name in os.listdir(destination_db) if os.path.isfile(
        # os.path.join(destination_db, name))])
# NOTE: depending on size of folder, this can error out multiple times - 
#Colab will cache the results, so retry until success and log the number for future attempts.
# print(dest_datasize)

In [ ]:
def chunk_reader(fobj, chunk_size=1024):
    while True:
        chunk = fobj.read(chunk_size)
        if not chunk:
            return
        yield chunk


def get_hash(filename, first_chunk_only=False, hash=hashlib.sha1):
    hashobj = hash()
    file_object = open(filename, 'rb')

    if first_chunk_only:
        hashobj.update(file_object.read(1024))
    else:
        for chunk in chunk_reader(file_object):
            hashobj.update(chunk)
    hashed = hashobj.digest()

    file_object.close()
    return hashed


def check_for_duplicates(path, hash=hashlib.sha1):
    hashes_by_size = defaultdict(list)  # dict of size_in_bytes: [full_path_to_file1, full_path_to_file2, ]
    hashes_on_1k = defaultdict(list)  # dict of (hash1k, size_in_bytes): [full_path_to_file1, full_path_to_file2, ]
    hashes_full = {}   # dict of full_file_hash: full_path_to_file_string
    counter = 0


    for filename in os.scandir(destination_db):
        counter += 1
        if (counter % 1000 == 0): print("completed " + str(counter) + " of " + str(dest_datasize))
        full_path = destination_db + "/" + str(filename.name)
        file_size = os.path.getsize(full_path)
        if (file_size < min_size): 
          print("removing" + filename)
          os.remove(filename)

        if (file_size >= min_size): hashes_by_size[file_size].append(full_path)

    counter = 0

    for size_in_bytes, files in hashes_by_size.items():

        if len(files) < 2:
            continue 

        for filename in files:
          counter += 1
          if (counter % 1000 == 0): print("completed hash for" + str(counter) + " of " + str(len(hashes_by_size)))
          small_hash = get_hash(filename, first_chunk_only=True)
          hashes_on_1k[(small_hash, size_in_bytes)].append(filename)

    for __, files_list in hashes_on_1k.items():
        if len(files_list) < 2:
            continue

        for filename in files_list:
            try: 
                full_hash = get_hash(filename, first_chunk_only=False)
                duplicate = hashes_full.get(full_hash)
                if duplicate:
                    print("Duplicate found: {} and {}".format(filename, duplicate))
                    os.remove(filename)
                else:
                    hashes_full[full_hash] = filename
            except (OSError,):
                # the file access might've changed till the exec point got here 
                continue


if __name__ == "__main__":
      check_for_duplicates("/content/drive/My Drive/IRCMS_GAN_collaborative_database/Experiments/colab-violingan/archon-analysis")